In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

In [2]:
df_original = pd.read_csv('../clean_dataset.csv', sep=',', header=0)
df = pd.DataFrame(data=df_original)
sns.set(style="ticks", color_codes=True)

### Divisão das informações para treinamento e teste

In [3]:
features = df[[c for c in df if c != "shares"]]
target = df["shares"]

X_train, X_test, Y_train, Y_test = train_test_split(features, target, test_size=0.3, random_state=12)

# RandonForest


É um algoritmo de aprendizagem de máquina flexível e fácil de usar que produz excelentes resultados a maioria das vezes, mesmo sem ajuste de hiperparâmetros

### Funcionamento

Esse algoritmo cria uma “floresta” que é uma combinação de árvores de decisão, na maioria dos casos treinados com o método de bagging. A idéia principal do método de bagging é que a combinação dos modelos de aprendizado aumentando o resultado geral.


Sendo mais claro o algoritmo RandonFlorest cria várias árvores de decisão e as combina para obter uma predição com maior acurácia e mais estável.

O algoritmo de RandonFlorest adiciona aleatoriedade extra ao modelo quando está criando as árvores. Ao invés de procurar pela melhor característica ao fazer a partição de nós, ele busca a melhor característica em um subconjunto aleatório das características. Este processo cria uma grande diversidade, o que geralmente leva a geração de modelos melhores.



## Importância das caracteristicas

Outra grande qualidade do RandonForest é a facilidade para se medir a importância relativa de cada característica (feature) para a predição. 


Sklearn provê uma excelente ferramenta para isto, que mede a importancia das características analisando quantos nós das árvores, que usam uma dada característica, reduzem impureza geral da "floresta". Ele calcula este valor automaticamente para cada característica após o treinamento e normaliza os resultados para que a soma de todas as importancias seja igual a 1.


Através da inspeção da importância das características, você pode decidir quais características deixar de fora do modelo, já que eles não contribuem o suficiente ou nada para o processo de predição. Isto é importante, porque uma regra geral em aprendizagem de máquina é que quanto mais características você tem, mais provavelmente seu modelo irá sofrer de superajuste (overfitting) e vice versa.


## Diferenças


Se você treinar uma árvore de decisão com um dataset de treinamento e rótulos, ela vai elaborar um conjunto de regras que serão utilizadas para realizar predições.

Por exemplo, se você quiser prever se uma pessoa vai clicar em um anúncio online, você pode colecionar anúncios que a pessoa clicou no passado e algumas características que descrevam esta decisão. Se você colocar as características e rótulos em uma árvore de decisão ela vai gerar nos e algumas regras. Você poderá então prever ser o anúncio será clicado ou não. Quando uma árvore de decisão gera regras e nodos, normalmente utiliza calculos de ganho de informação. Por outro lado, a RandonFlorest faz isto de modo aleatório.

Outra diferença é que árvores de decisão profundas podem sofrer de sobreajuste (overfitting). RandonFlorest evitam o sobreajuste na maioria dos casos, pois trabalha com subconjuntos aleatórios das características e constrói árvores menores a partir de tais subconjuntos. Depois do treinamento, as subárvores são combinadas. Esta abordagem torna a computação mais lenta, dependendo de quantas árvores serão construiídas pelo Floresta Aleatória.


## Hiperparâmetros Importantes

### Aumentando o poder de predição
**n_estimators**
, que indica o número de árvores construídas pelo algoritmo antes de tomar uma votação ou fazer uma média de predições. Em geral, uma quantidade elevada de árvores aumenta a performance e torna as predições mais estáveis, mas também torna a computação mais lenta.

**max_features**, que indica o número máximo de características a serem utilizadas pelo Floresta Aleatória na construção de uma dada árvore. Sklearn oferece várias opções, descritas na sua documentação.

**min_sample_leaf** Este parâmetro indica o número mínimo de folhas que devem existir em uma dada árvore.

### Aumentando a velocidade do modelo
**n_jobs** informa quantos processadores o algoritmo pode utilizar. Se ele tiver valor 1, pode utilizar apenas um processador. O valor -1 significa que não há limite na quantidade de processadores a ser utilizada.

**random_state** torna o resultado do modelo replicável. O modelo será produzido do mesmo modo se ele tiver um valor definido de random_state e se forem utilizados os mesmos parâmetros com o mesmos dados de treinamento.

**oob_score**(também chamado de oob sampling), que é um método de validação cruzada para floresta aleatória. Neste tipo de amostragem (sampling), cerca de um terço dos dados não é utilizado no treinamento e pode ser utilizado para avaliar a performance. Estas amostras são chamadas out of the bag samples. É uma técnica similar ao método de validação cruzada leave one out, mas sem nenhum custo computacional extra.

## Resumo

- Muito bom para primeiros estágios de criação de desenvolvimento de um modelo
- Provê um bom indicador de importância para as características;
- Apresenta boa perfomance;
- É possível trabalhar com diferentes tipos de entrada

geralmente rápida, simples e flexível embora ainda apresente limitações

In [4]:
from sklearn.ensemble import RandomForestRegressor

def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy


regr =  RandomForestRegressor(max_depth=100,max_features=30, random_state=0, 
                              min_samples_leaf=60 ,min_samples_split=20,
                              n_estimators=100,n_jobs=3)

regr.fit(X_train, Y_train)

base_accuracy = evaluate(regr, X_test, Y_test)

print("\nFeature Importance\n")
features_importance = zip(regr.feature_importances_, features)
for importance, feature in sorted(features_importance, reverse=True):
    print("%s: %f%%" % (feature, importance*100)) 

Model Performance
Average Error: 1833.7951 degrees.
Accuracy = -13.50%.

Feature Importance

kw_avg_avg: 23.111661%
kw_max_avg: 7.398560%
self_reference_avg_sharess: 7.041871%
self_reference_min_shares: 6.657751%
LDA_04: 3.195387%
kw_avg_max: 2.707193%
Unnamed: 0: 2.564403%
LDA_01: 2.384527%
num_imgs: 2.198640%
global_subjectivity: 2.138951%
kw_min_avg: 2.093098%
average_token_length: 2.088474%
kw_avg_min: 2.049406%
num_hrefs: 2.015787%
n_unique_tokens: 1.986279%
Unnamed: 0.1: 1.928028%
self_reference_max_shares: 1.800752%
LDA_00: 1.786278%
n_non_stop_unique_tokens: 1.777405%
LDA_02: 1.670280%
kw_max_min: 1.632339%
n_tokens_content: 1.603699%
avg_negative_polarity: 1.582350%
LDA_03: 1.547931%
global_sentiment_polarity: 1.464608%
title_subjectivity: 1.419140%
avg_positive_polarity: 1.347295%
global_rate_positive_words: 1.271624%
global_rate_negative_words: 1.143909%
kw_min_max: 1.104748%
abs_title_sentiment_polarity: 1.039361%
title_sentiment_polarity: 0.887242%
rate_negative_words: 0.7

In [5]:
# Passando outros parametros

regr =  RandomForestRegressor(max_depth=13, random_state=0,n_estimators=100,n_jobs=6,)

regr.fit(X_train, Y_train)

base_accuracy = evaluate(regr, X_test, Y_test)

print("\nFeature Importance\n")
features_importance = zip(regr.feature_importances_, features)
for importance, feature in sorted(features_importance, reverse=True):
    print("%s: %f%%" % (feature, importance*100)) 

Model Performance
Average Error: 1867.6129 degrees.
Accuracy = -17.28%.

Feature Importance

kw_avg_avg: 11.363578%
kw_max_avg: 3.630947%
self_reference_min_shares: 3.528921%
self_reference_avg_sharess: 3.474756%
average_token_length: 3.292514%
LDA_01: 3.198821%
LDA_04: 3.167312%
kw_avg_max: 3.127554%
avg_positive_polarity: 2.986008%
global_subjectivity: 2.898667%
LDA_03: 2.765580%
n_tokens_content: 2.675234%
LDA_00: 2.635473%
global_sentiment_polarity: 2.633790%
n_unique_tokens: 2.601172%
kw_avg_min: 2.579613%
num_hrefs: 2.577746%
n_non_stop_unique_tokens: 2.530502%
LDA_02: 2.488869%
kw_min_avg: 2.324815%
kw_max_min: 2.302124%
avg_negative_polarity: 2.294652%
Unnamed: 0.1: 2.252674%
global_rate_positive_words: 2.251337%
num_imgs: 2.097454%
global_rate_negative_words: 2.057286%
Unnamed: 0: 2.037663%
n_tokens_title: 1.978580%
title_sentiment_polarity: 1.908732%
self_reference_max_shares: 1.764882%
max_negative_polarity: 1.506894%
kw_min_max: 1.410838%
num_self_hrefs: 1.401676%
title_sub

## Técnica de Grid Search

Vimos que os melhores valores para esses parâmetros mudam conforme os dados mudam, conforme você adiciona ou tira features e conforme muda os outros parâmetros também.

Mas então, como definir quais os melhores valores? 

Uma das técnicas que pode ser utilizada para isso é o Grid Search CV. 

Você passa para ele uma lista de possíveis valores e o score usado para medir a eficiência do modelo, ele vai rodar o Cross Validation com todas as possíveis combinações e no final vai te dizer qual a combinação apresentou o melhor score.




In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [5, 10, 15],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [20, 30, 40, 35]
}

grid_search = GridSearchCV(estimator = regr, param_grid = param_grid,cv = 3, n_jobs = 3, verbose = 2)


grid_search.fit(X_train, Y_train)

rgr_rf = grid_search.best_estimator_ 
grid_search.best_params_, grid_search.best_score_

Fitting 3 folds for each of 216 candidates, totalling 648 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed:    7.5s
[Parallel(n_jobs=3)]: Done 156 tasks      | elapsed:   29.2s
[Parallel(n_jobs=3)]: Done 359 tasks      | elapsed:  1.2min
[Parallel(n_jobs=3)]: Done 642 tasks      | elapsed:  2.3min
[Parallel(n_jobs=3)]: Done 648 out of 648 | elapsed:  2.4min finished


({'max_depth': 10,
  'max_features': 3,
  'min_samples_leaf': 5,
  'min_samples_split': 12,
  'n_estimators': 40},
 0.07191071144364523)